<a href="https://colab.research.google.com/github/wrightky/ANUGA_DXWorkshop/blob/main/WLAD_Model/WLAD_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<div align="center">**Wax Lake & Atchafalaya Delta Model**</div>
##<div align="center">ANUGA Tutorial</div>
##<div align="center">Kyle Wright</div>
###<div align="center">Delta-X Applications Workshop (May 5th, 2022)</div>

## Information on this session:

**Goal:** Introduce the basics of the hydrodynamic model for the Wax Lake & Atchafalaya Delta (WLAD) basin using ANUGA. This model solves the 2D shallow-water equations on an unstructured triangular grid. Note that the model structure has been slightly simplified for parts of this demonstration.

**Software:** ANUGA, Python, Jupyter Notebooks, other packages

**Installation:** None, this lab is run on the cloud in the Google `colaboratory` environment.

**Details:** In this tutorial, we will build, run, and analyze our model of the actively-aggrading WLAD basin. This is one of the two Delta-X study sites, and we will be simulating the period that aligns with the Spring 2021 campaign. Different cell blocks will demonstate how one can build the model mesh, initialize the domain, setup the parameters of the model, and run the simulation. 

The regular version of this model is written as a collection of Python scripts that are run in parallel on a high-performance computing cluster (e.g. [TACC](https://www.tacc.utexas.edu/)). However, we have simplified the model here to allow it to run in this notebook on Google Colab, so that we can demonstrate the workflow of the model, which is identical to that in the true version. The raw codes for the current parallel implementation have been included in this repository along with this notebook, so feel free to glance through those as well. 

This notebook is written in Python 3, and has been designed to run in the Google `colaboratory` environment, which provides a Jupyter notebook environment running on a virtual machine on the cloud. Nothing needs to be installed locally, as it is entirely run on Google Drive. All you need is a Google account. All of these scripts are stored on GitHub.

To start interacting with the notebook, make sure it is open in Colab (see `Open in Colab` link above). Once you're there, click `File` > `Save a Copy in Drive`.

## Setup Environment

The first thing we have to do is set up our environment. In Python, this means downloading all the necessary functions we want to use, including ANUGA.

Run the following cell to install the dependencies and some extra code for visualising on Colaboratory. The install should take less than a minute.

*Pro-tip: To run each code block, you can simply type `shift` + `enter`*

NOTE: Tutorial installation is based on the [2018 CSDMS Anuga Clinic](https://github.com/stoiver/anuga-clinic-2018), but updated to work in Python 3.

In [1]:
# Here we download/install all the files we need behind the scenes
try:
    import os
    os.chdir('/content')
    # Grab workbook files into colab directory
    !git clone https://github.com/wrightky/ANUGA_DXWorkshop.git
    # Install everything using some bash scripts
    !/bin/bash /content/ANUGA_DXWorkshop/anuga_tools/install_anuga_colab.sh
    !python /content/ANUGA_DXWorkshop/anuga_tools/install.py > /dev/null 2>&1 
except:
    pass

# Deal with some directory stuff
if not 'workbookDir' in globals():
    workbookDir = os.getcwd()
import sys
sys.path.append(os.path.join(workbookDir,"ANUGA_DXWorkshop"))
sys.path.append(os.path.join(workbookDir,"anuga_core"))

Cloning into 'ANUGA_DXWorkshop'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (229/229), done.
remote: Total 249 (delta 22), reused 240 (delta 17), pack-reused 0
Receiving objects: 100% (249/249), 2.24 MiB | 6.67 MiB/s, done.
Resolving deltas: 100% (22/22), done.
(1) Install pip packages
(2) Install gdal
(3) Install netcdf4
(4) Download anuga_core github repository
(5) Install anuga
(6) Ready to go


## Import packages

Now that everything is installed, we can load the packages we want to use directly into this notebook. Most of these are very common packages, and each one contains a different group of functions that are useful for scientific applications. We access these functions by 'importing' the packages we need. See the comments below for a brief discription of what each of these packages are for.

In [2]:
import numpy as np # Array manipulation, most important Python package
import matplotlib # Plotting/visualization
import matplotlib.pyplot as plt # Convenient shorthand for the main plotting function
matplotlib.use('Agg') # Settings
%matplotlib inline
from scipy.interpolate import interp1d # Simple linear interpolation
import pandas as pd # Spreadsheets/excel data
import os # Operating system, handles directories
import glob # Also for files
# from PIL import Image # For making output animations
import anuga # The model itself
from anuga.utilities import animate # Built-in plotting tool for the model
from anuga import Inlet_operator # Explained later

## Create ANUGA domain and mesh

Now we can begin constructing our model domain. This means we need to create a mesh, add any initial and boundary conditions, and handle any settings.

First, we need to specify our model geometry. In the following cell, we specify the coordinates of the boundary polygon for our model, and **instantiate the model domain and mesh** from that polygon. The internal mesh engine does almost all of the work for us here, we just need to feed it our **domain size**, desired **mesh resolution**, and any **interior polygons** we might want (if we have any). 

Interior polygons are one of the powerful features of ANUGA. We can have any number of regions inside our domain with their own mesh resolution. Is there an area you really care about and want to model in high-resolution? Easy. Is there an area you need to include but don't care a lot about, and can make very coarse to save computational time? Also easy. The main constraint is that the polygons defining the boundary of those regions can't overlap.

For visualization, we will be making use of ANUGA's `animate.Domain_plotter` tool, which we initialize here and call repeatedly later.

*Note: Always make sure units are in metric, ANUGA does not support customary units*

In [ ]:
# ---------------Define Domain Boundary-----------------
bounding_polygon = [[622338.0, 3339415.0], [623005.0, 3341128.0],
                    [626983.0, 3340039.0], [626675.0, 3338530.0]]

# Assign a name to the different kinds of boundaries, needed for BCs later
boundary_tags={'sides': [0,1,3], 'downstream': [2]} # ID of segments, not points

# ---------------Define Geo Reference-------------------
geo_reference = anuga.Geo_reference(zone=14,
                                    datum='wgs84',
                                    projection='UTM',
                                    false_easting=500000,
                                    false_northing=0)

# Mesh Resolution (max area of triangles in each given region)
base_res = 625. # Ensures that all cells have less area than this max value

# ---------------Generate domain and mesh---------------
domain = anuga.create_domain_from_regions(bounding_polygon, boundary_tags,
                                          maximum_triangle_area=base_res,
                                          mesh_geo_reference=geo_reference,
                                          mesh_filename='onion.msh')

# Plot mesh
fig = plt.figure(figsize=(5, 5), dpi=250, facecolor='w', edgecolor='k')
dplotter = animate.Domain_plotter(domain)
plt.triplot(dplotter.triang, linewidth=0.1);
plt.axis('scaled')

# Print out some descriptive statistics of our mesh cells
print(domain.statistics())

The previous cell should print out some statistics of your mesh, as well as a picture of it. Note the "Number of triangles". How does this value change if you modify the `base_res` (mesh resolution)?

After this cell, our model `domain` now exists, and we can begin populating it with information about our study site.

## Create bathymetry/topography

Now that we have a mesh, we need to provide **elevation** values for each of those cells. We do this using a **topographic DEM** stored in an ASCII file. This is the same DEM used in Lab 3, just converted into a different format and metric units. Because ANUGA requires DEMs to be stored in certain data-types, here we convert this data a couple times until it's in a format ANUGA can use (`.dem`, `.pts`). Finally, we use the resulting `.pts` file to create our model bathymetry.

These steps can be **quite slow** for high-resolution meshes, so if the mesh isn't changing between runs, we recommend **saving the resulting topography** from the domain as a `.csv` and just re-loading that file for future runs. This file will be a long list of elevation values sorted by triangle ID.

*Note: The ASCII file needs to have the same headers as the file used here, which sometimes isn't the default. To repeat these steps, check the ASCII file with a text editor and make sure it has the same headers as the example ASCII files in th ANUGA examples folder. It also needs the accompanying `.prj` file!*

In [ ]:
# ASCII to DEM conversion
anuga.asc2dem('ANUGA_OnionCreek_Tutorial/data/EG_OnionCreek_2m_EPSG_32614_m.asc',
              use_cache=False, verbose=True)
# DEM to "points"
anuga.dem2pts('ANUGA_OnionCreek_Tutorial/data/EG_OnionCreek_2m_EPSG_32614_m.dem',
              use_cache=False, verbose=True)
# Use "points" to populate mesh cells
domain.set_quantity('elevation', 
                    filename='ANUGA_OnionCreek_Tutorial/data/EG_OnionCreek_2m_EPSG_32614_m.pts',
                    use_cache=False, verbose=True, alpha=0.1)

# Plot topography
fig=plt.figure(figsize=(5, 5), dpi= 250, facecolor='w', edgecolor='k')
plt.tripcolor(dplotter.triang, facecolors = dplotter.elev, cmap = 'terrain')
plt.colorbar(fraction=0.02);
plt.title("Elevation");
plt.axis('scaled')

# To save time next time, we can save the result
# topo = domain.quantities['elevation'].centroid_values
# np.savetxt('ANUGA_OnionCreek_Tutorial/data/EG_OnionCreek_2m_EPSG_32614_m.csv', topo, delimiter=",")

## Add boundary conditions

Now we use some nearby USGS gauge data to prescribe our model **boundary conditions**. Here we use two common types of BCs: **Dirichlet** and **Reflective**. For the former, we tell the model exactly what the water surface elevation and/or velocities are along that boundary as a function of time. For the latter, the boundary acts like a solid wall, bouncing any flows back inside the domain. 

We assign our downstream boundary condition to match the water surface elevations of the nearby gauge using ANUGA's `Time_boundary` function. ANUGA has **three conserved quantities** that we have to specify: [`stage`, `xmomentum`, `ymomentum`]. `stage` is the water surface elevation ($\eta = z + h$), and `momentum` $q = [uh, vh]$. We use the gauge data to specify these along the downstream BC as a function of time (only `stage` is set to vary here). All the other boundaries are designated as a no-flux `Reflective_boundary`, because we don't expect flows in those directions.

*Note: Due to **poor boundary placement** in relation to where these gauges are located, we've added a few fudge factors in the following sections for the purposes of this demonstration. Ideally, your boundaries should be drawn where you have a good sense for the **mass balance** of your system!*

In [ ]:
# Load gauge data into a Pandas "DataFrame"
usgs_a = pd.read_csv('ANUGA_OnionCreek_Tutorial/data/USGS_Onion183_metric.csv',
                     header=0, names=['ID','datetime','tz','Q','WL'])
usgs_a['datetime'] = pd.to_datetime(usgs_a['datetime']) # Convert to datetime dtype
# Need to convert datetimes into 'seconds from initial time'
usgs_a['epoch'] = (usgs_a['datetime'] - usgs_a['datetime'][0]) // pd.Timedelta("1s")

usgs_b = pd.read_csv('ANUGA_OnionCreek_Tutorial/data/USGS_Williamson_metric.csv',
                     header=0, names=['ID','datetime','tz','Q','WL'])
usgs_b['datetime'] = pd.to_datetime(usgs_b['datetime']) # Convert to datetime dtype
# Need to convert datetimes into 'seconds from initial time'
usgs_b['epoch'] = (usgs_b['datetime'] - usgs_b['datetime'][0]) // pd.Timedelta("1s")

# Print out one of these DataFrames to get a sense for what it looks like:
usgs_a # This will print out a snippet. Looks like a spreadsheet!

Now we have loaded in the data from two USGS gauges. Gauge "A" is [08159000 on Onion Creek at 183](https://waterdata.usgs.gov/nwis/uv/?site_no=08159000&agency_cd=USGS) and gauge B is [08158970 on Williamson Creek](https://waterdata.usgs.gov/nwis/uv/?site_no=08158970&agency_cd=USGS). Technically Gauge A is in the middle of our domain, but for this demonstration we're going to use it to define an inflow.

The two datasets correspond to the storm that came through on March 21st 2022, which is the time period we're going to be modeling. The downloaded files span from 5PM on March 21st through March 24th, and all values have been converted to $[m]$ or $[m^3/s]$

Let's use the water level information from Gauge A to specify the `stage` for our downstream boundary, by creating a function that interpolates this `DataFrame` as a function of time:

In [ ]:
vert_offset = 1.0 # Fudge factor, vertical correction to gauge data
# Create a linear interpolation function for the WSE:
fBC_downstream = interp1d(usgs_a['epoch'], usgs_a['WL'] + vert_offset, kind='linear')

# Downstream boundary
Bout = anuga.Time_boundary(domain, function=lambda t: [fBC_downstream(t), 0.0, 0.0])

# All other boundaries
Br = anuga.Reflective_boundary(domain)

# Assign BCs to the boundaries we named earlier
domain.set_boundary({'downstream': Bout, 'sides': Br})

## Add discharge hydrograph

Now we add inflows of **discharge** from each of the two upstream channels. We do this by creating an **inlet** through which that discharge is added to the model.

The way ANUGA handles discharge is somewhat odd compared to other models. Most models define a mass inflow as a boundary condition, and every time-step mass is added appropriately. Due to some weird quirks about how ANUGA's numerical solver handles BCs, it works better to add an **inlet** *inside the domain* through which water flows into our domain, somewhat like a flooding bath drain. (Note that **outlets** can be created the same way by specifying a **negative discharge**)

In [ ]:
# Create two more linear interpolation functions for Q
# Fudge factor, changing timing & magnitude of gauge A to make up for poor BC placement
usgs_a['epoch_2'] = (usgs_a['datetime'] - usgs_a['datetime'][0] - pd.Timedelta('70 min')) // pd.Timedelta("1s") # Moves up flood arrival by 70 min
fQ_a = interp1d(usgs_a['epoch_2'], usgs_a['Q']*0.8, kind='linear') # Scales down Q by 20%
fQ_b = interp1d(usgs_b['epoch'], usgs_b['Q'], kind='linear')

# Setup inlets (supply coordinates + add "Inlet_operator")
first_inlet = np.array([[622627.,3339957.],[622627.,3339982.]]).astype('double') # Coordinates of line
first_inflow = Inlet_operator(domain, first_inlet, Q = fQ_a)

second_inlet = np.array([[623031,3341074],[623035,3341085]]).astype('double') # Coordinates of line
second_inflow = Inlet_operator(domain, second_inlet, Q = fQ_b)

# Visualize time-varying Q
tt = np.linspace(0., 86400, 1000) # Vector of time values for plotting
Qplta = [fQ_a(ti) for ti in tt] # Vector of Q values for each tt
Qpltb = [fQ_b(ti) for ti in tt] # Vector of Q values for each tt
plt.figure(figsize=(6,3), dpi=150)
plt.plot(tt/3600., Qplta, 'g-')
plt.plot(tt/3600., Qpltb, 'b-')
plt.xlabel('Time [hrs]')
plt.ylabel('Discharge [$m^3/s$]')
plt.legend(['Inlet A','Inlet B'])

Now our domain will have inflows when we run the code. The `Inlet_operator` that we used above is an optional add-on function that tells ANUGA, in plain language, "*there is an inlet here with this amount of discharge*". "Operator" is a generic term for lots of optional add-ons in ANUGA that can modify the model in some way. The location of our inlet is just a simple line, and all the cells that intersect that line will act as our inlet. 

## Add rainfall

We can also add rainfall to our model, which adds water to all our model cells at some specified rate (mm/s) over a specified period of time. 

For this simulation, we've simplified the real hyetograph into two windows of rainfall, based on nearby City of Austin estimates. There were roughly two periods of rainfall on March 21st: window 1 was around 6PM, and window 2 was around midnight. About 3.2 inches (81.3 mm) fell in total.

In [ ]:
# Details for our simplified hyetograph
window1 = [2700, 5400] # Seconds after model start time (5PM on March 21st)
window2 = [23400, 28800]
rain_rate_window1 = 0.006778 # mm/s
rain_rate_window2 = 0.011667 # mm/s

# Create a function that returns the rainfall rate as a function of time
def rainfall(t):
  if window1[0] <= t < window1[1]:
    rate = rain_rate_window1
  elif window2[0] <= t < window2[1]:
    rate = rain_rate_window2
  else:
    rate = 0.
  return rate

# Append to forcing terms; this adds rain to our model domain
R = anuga.Rainfall(domain, rate=rainfall)
domain.forcing_terms.append(R)

# Visualize time-varying R
tt = np.linspace(0., 86400, 1000) # Vector of time values for plotting
RR = [rainfall(ti) for ti in tt] # Vector of rainfall values for each tt
plt.figure(figsize=(6,3), dpi=150)
plt.plot(tt/3600., RR, 'k-')
plt.xlabel('Time [hrs]')
plt.ylabel('Rainfall rate [$mm/s$]')

## Initialize Stage

Now we tie up a few loose ends. We need to specify the **stage** initial condition, which we set to equal the height at the downstream boundary anywhere where the elevation is lower than that value (and dry everywhere else). We can do this using `set_quantity()`, which we could have used to initialize any of our variables if we wanted to (e.g. momentum)

In [ ]:
# ---------------Load pre-established elevation----------------------
topo = domain.quantities['elevation'].centroid_values
stage = topo.copy()  # Initialize stage as = topography
# Is elevation is lower than WSE downstream, set stage = WSE
stage[topo <= fBC_downstream(0)] = fBC_downstream(0)
domain.set_quantity('stage', stage, location='centroids')  # Initialize depth

## Initialize Friction

Finally, we apply a uniform Manning's coefficient $n=0.05$ everywhere in the domain. Note that this value can vary spatially, we're just using one value for simplicity.

In [ ]:
# Create empty vector of 1's with the same dimensions as topo, multiply by n
friction = np.ones_like(topo)*0.05
domain.set_quantity('friction', friction, location = 'centroids')

## Run the model

Finally, we can run the model!

Here we call on `domain.evolve()` to **run the model** for a specified amount of time. There are two kinds of timesteps to be aware of: the **model timestep** and the **yieldstep**. In ANUGA, the model timestep is automatically computed based on numerical stability conditions, and is usually some small value $\approx \mathcal{O}(1s)$. However, we can't save the data for every timestep, because output file would be enormous. We therefore specify a yieldstep, which is the time interval (in seconds) at which the model **saves information** on the model's converved quantities (`stage`, `xmomentum`, `ymomentum`) into the output file. We use the command `domain.print_timestepping_statistics()` to tell us some summary statistics of these timesteps every `yieldstep`.

As the model runs, output data are saved into a NetCDF file with the extension `.sww`. This is a common and efficient data type for storing numeric data, and can be opened by many programs, not just ANUGA. 

The `domain.evolve()` function works like a typical *for* loop: *for t less than final time, evolve the model*. This enables us to **make changes to our model** as time progresses, if we wish. In this example, all of our BCs were defined as functions of time, so they are automatically updated each time step, but they also could have been defined as static values and then updated inside this loop.

In [ ]:
# ------------------------------------------------------------------------------
# Evolve system through time
# ------------------------------------------------------------------------------
savefreq = 900. # Yieldstep, in seconds
simtime = 86400. # Total simulation time, in seconds

for n, t in enumerate(domain.evolve(yieldstep=savefreq, finaltime=simtime)):
    # Print out some statistics on the model evolution
    domain.print_timestepping_statistics()

As the model evolves, you should see timestepping statistics printed out every yieldstep. Here's the meaning for those numbers:
- The first value is the "real" amount of time that has been modeled so far.
- "delta t" is the range of model timesteps taken during that yieldstep.
- "steps" is the number of model timesteps taken in that yieldstep.
- The number in the parentheses at the end is the amount of clock time that has elapsed since the last yieldstep, i.e. how long you'll have to wait for the model to finish.

## Inspect outputs

Now that we've run the model, let's view some results!

There is also a pre-saved output file in the GitHub folder, which you can use if you don't want to wait for your run to finish.

There are a couple ways one can query the output file. ANUGA's `.sww` files can be loaded using any normal NetCDF reader, but ANUGA also comes pre-packaged with several functions for inspecting your model results, most of which are stored in `anuga.utilities.plot_utils`.

First, load in the data:

In [ ]:
# Uncomment whichever one we're visualizing:
saved_output = 'domain.sww' # Output of current run
# saved_output = 'ANUGA_OnionCreek_Tutorial/notebooks/domain.sww' # Pre-saved output

# Load in values of conserved quantities at every cell centroid
swwvals = anuga.utilities.plot_utils.get_centroids(saved_output, timeSlices='all')
# Query values: time, x, y, stage, elev, height, xmom, ymom, xvel, yvel, friction, vel, etc
model_starttime = pd.to_datetime('2022-03-21 17:00:00')
dplotter = animate.SWW_plotter(saved_output) # Update dplotter if necessary

# Separate variables just for accessibility
time = swwvals.time
topo = swwvals.elev
x = swwvals.x
y = swwvals.y
depth = swwvals.height
stage = swwvals.stage
u = swwvals.xvel
v = swwvals.yvel
w = np.sqrt(u**2 + v**2)

Figure files for each frame will be stored in _plot


## Make plotting functions

To clean up the code, let's wrap some of these plotting codes into functions. Run the hidden block below, and feel free to open it if you want to see the details.

In [ ]:
def show_velocity(time_idx):
  '''Makes figure of flow velocity at some time index'''
  timeCDT = pd.to_timedelta(time[time_idx], 's') + model_starttime # Find time
  fig = plt.figure(figsize=(5, 5), dpi=250, facecolor='w', edgecolor='k')
  plt.tripcolor(dplotter.triang, facecolors=w[time_idx,:], 
                vmax=0.3, vmin=0, cmap='plasma') # Plot velocities
  cbar = plt.colorbar(fraction=0.02); # Add colorbar
  cbar.set_label('Velocity [m/s]', rotation=270, labelpad=10)
  plt.title('Velocity : '+timeCDT.strftime("%b %d %Y, %H:%M"));
  plt.axis('scaled') # Scale axes
  return

def show_quiver(time_idx):
  '''Makes quiver plot of flow velocity at some time index'''
  timeCDT = pd.to_timedelta(time[time_idx], 's') + model_starttime # Find time
  # Indices of where water is located:
  inds = (w[time_idx,:] > 0.05) & (depth[time_idx,:] > 0.01)
  inds[0::2] = 0  # Cut it down to half as many arrows
  fig = plt.figure(figsize=(8, 8), dpi=300, facecolor='w', edgecolor='k')
  # Show topography as the background:
  plt.tripcolor(dplotter.triang, facecolors=dplotter.elev, cmap='Greys_r')
  plt.colorbar(fraction=0.01);
  plt.title("Flow Direction : "+timeCDT.strftime("%b %d %Y, %H:%M"));
  # Create the quivers:
  Quiv = plt.quiver(x[inds], y[inds], u[time_idx,inds], v[time_idx,inds],
                    color='red', scale=30, width=0.001, headwidth=4, pivot='mid')
  qk = plt.quiverkey(Quiv, 0.18, 0.66, 1, r'$1 \frac{m}{s}$', labelpos='W',
                    coordinates='figure', labelcolor='red') # Add a key
  plt.axis('scaled') # Scale axis
  return

def show_inundation(time_idx):
  '''Makes figure of flow depth at some time index'''
  timeCDT = pd.to_timedelta(time[time_idx], 's') + model_starttime # Find time
  fig = plt.figure(figsize=(5, 5), dpi=250, facecolor='w', edgecolor='k')
  # Show topography as the background:
  plt.tripcolor(dplotter.triang, facecolors=topo, cmap='Greys_r')
  # We need to copy the variable so we can mask-out shallow cells:
  plot_depth = depth[time_idx,:].copy()
  plot_depth[plot_depth < 0.01] = np.nan
  plt.tripcolor(dplotter.triang, facecolors=np.log10(plot_depth),
                vmin=-2, vmax=1, cmap='Blues') # Plot depths
  cbar = plt.colorbar(fraction=0.02) # Add colorbar
  cbar.set_label('log10(Depth) [m]', rotation=270, labelpad=10)
  plt.title('Depth : '+timeCDT.strftime("%b %d %Y, %H:%M"));
  plt.axis('scaled') # Scale axis
  return

def show_max_inundation():
  '''Makes a figure of maximum inundation'''
  fig = plt.figure(figsize=(5, 5), dpi=250, facecolor='w', edgecolor='k')
  # Show topography as the background:
  plt.tripcolor(dplotter.triang, facecolors=topo, cmap='Greys_r')
  # We need to copy the variable so we can mask-out shallow cells:
  plot_depth = np.nanmax(depth, axis=0) # Find max
  plot_depth[plot_depth < 0.01] = np.nan
  plt.tripcolor(dplotter.triang, facecolors=np.log10(plot_depth),
                vmin=-2, vmax=1, cmap='Blues') # Plot depths
  cbar = plt.colorbar(fraction=0.02) # Add colorbar
  cbar.set_label('log10(Max Depth) [m]', rotation=270, labelpad=10)
  plt.title('Maximum Depth');
  plt.axis('scaled') # Scale axis
  return

## Flow Velocity

One of the strengths of 2D modeling is getting to see the spatial patterns of flow in our landscape. Let's visualize the flow velocity at some particular time.

Change `time_idx` to view the flow field at different times.

In [ ]:
# Time index to plot:
time_idx = 32 # integer from 0-94

# Make the figure
show_velocity(time_idx)

We can also visualize these velocities using a "quiver map", with arrows showing the direction of flow.

Because there are a lot of arrows, we use the variable `inds` below to only show arrows in relevant cells.

In [ ]:
# Time index to plot:
time_idx = 32

# Make the figure
show_quiver(time_idx)

## Inundation

We can also visualize the flow depth, which is useful for monitoring flooding. In the cells below, we filter out depths $< 1cm$ to show flooding more clearly. We also plot the color-axis on a log-scale so you can more easily see spatial differences (the range $[-2, 1] = [1cm, 10m]$). 

In [ ]:
# Time index to plot:
time_idx = 35

# Make the figure
show_inundation(time_idx)

We can also plot the maximum inundation experienced by each cell:

In [ ]:
# Make the figure
show_max_inundation()

## Plot Gauge

We can also extract time-series information at a point, which can be useful if we have gauges inside our domain to compare to. Let's pull out the time-series of water surface elevations near our Gauge A's true location.

If this model is performing well, this should look similar to the water levels measured at that gauge. Let's plot both together and compare them.

In [ ]:
# Create triangle lookup function to find cell index of a gauge
p = anuga.utilities.plot_utils.get_output(saved_output)
tri_lookup = anuga.utilities.plot_utils.get_triangle_lookup_function(p)

# Create time vector of model outputs
model_timeseries = model_starttime + pd.to_timedelta(time, 's')

# Find coordinates of gauge and cell containing it, extract stage
xx, yy = [626300, 3339250]
idx = tri_lookup(xx, yy)
gaugestage = stage[:, idx]

# Create figure:
fig = plt.figure(figsize=(8, 4), dpi=150, facecolor='w', edgecolor='k')
plt.plot(model_timeseries, gaugestage, 'r--') # Plot modeled gauge
# Grab gauge stage from our downstream BC function
tt = np.linspace(0., 86400, 1000) # Vector of seconds values
gauge = [fBC_downstream(ti) for ti in tt] # Vector of WL values for each tt
gauge_timeseries = model_starttime + pd.to_timedelta(tt, 's') # Convert to time
plt.plot(gauge_timeseries, gauge, 'k-') # Plot gauge
plt.ylabel('Stage [m]')
plt.legend(['Modeled Stage','Gauge Stage'])

The fit could be better. If we wanted to try to improve it, our main options would probably be to: 
1. Change friction coefficients
2. Increase our mesh resolution
3. Move our model boundaries to better locations
4. Check our input data to make sure there aren't any errors (e.g. mis-matching datums)


## Plot Animation

Finally, instead of viewing snapshots of time, we could also use some of the functions above to make animations for the whole simulation. Let's re-run one of those functions, but this time, let's loop through all values of `time_idx`, save each output, and turn them into an animation.

In [ ]:
for time_idx in range(0,len(time)):
  # Make the figure for this time
  show_velocity(time_idx)
  # Save the figure and close it
  plt.savefig('figure_%s.jpg' % time_idx)
  plt.close()

# Grab the list of files, and make sure they're in order
files = glob.glob('figure_*.jpg')
files = sorted(files, key=os.path.getmtime)
outfile = 'animation.gif'

# Create GIF
imgs = [Image.open(f).copy() for f in files]
imgs[0].save(fp=outfile, format='GIF', append_images=imgs[1:],
              save_all=True, duration=200, loop=0) # Write gif

Once the above code has been run, you can view the animation by opening the file manager on the left-hand side panel (looks like a folder) and double-clicking the `animation.gif`. This should open the file in a right-hand side panel. 

**Bonus**: Change out the `show_velocity` function used inside the loop for one of the other functions, `show_quiver` or `show_inundation`, to make a different animation.